# 인공신경망 정함성 검증 Gradient Checking

당신이 세계적으로 사용가능한 모바일 결제 시스템을 만드는 팀이라고 상상해봅시다. 그리고 딥러닝을 통해 사기를 감지할 수 있도록 만들어야한다고 생각해봅시다. 거래가 발생할때마다 사기인지 정상적인 거래인지 감지하고 싶을 겁니다.

어느정도 정확도를 도출했을때 CEO 입장에서는 정확도가 나왔는데 이정확도가 어떤 수식으로 나왔고 올바른지 확인하고 싶을 겁니다.

이때 우리는 CEO에게 확신을 주기 위해 우리는 "gradient checking"을 사용해야합니다.

In [3]:
# Packages
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1) Gradient checking 은 어떻게 구하나요?

역전파는 각 매개변수 $\theta$에 대한 미분값 $\frac{\partial J}{\partial \theta}$을 계산하는 작업이 수행됩니다. (아시겠지만 $J$ 는 순전파의 결과물로 비용함수 값입니다.)  
우리는 $\frac{\partial J}{\partial \theta}$ 값들이 제대로 계산이 됐는지 확인할 필요가 있습니다.
  
순전파는 계산하기가 상대적으로 간단해서 $J$가 제대로 계산된건지 확인하는게 쉽습니다.  
그래서 $J$ 를 사용해 $\frac{\partial J}{\partial \theta}$가 올바르게 구해졌는지 알아볼겁니다.. 


경사도(gradient) 계산식의 정의를 다시 한번 확인해보죠:
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$


만약 "$\displaystyle \lim_{\varepsilon \to 0}$" 표현이 익숙하지 않다면 간단하게 $\varepsilon$ 가 작을 때를 의미합니다.


We know the following:

- $\frac{\partial J}{\partial \theta}$ 가 올바르게 계산됐는지 알고싶습니다. 
-  $J$ 값에 대해 확신할 수 있다면  $J(\theta + \varepsilon)$와 $J(\theta - \varepsilon)$ 를 구할 수 있습니다. (이때 $\theta$는 실수입니다.)


$\frac{\partial J}{\partial \theta}$ 를 통해 역전파 수식이 올바르게 작동하는것을 확인시켜줍시다.

## 2) 1차원 gradient checking

일단 간단하게 1차원 선형 수식을 생각해봅시다. (= $J(\theta) = \theta x$.) 모델은 매개변수 $\theta$ 1개를 사용하고 입력값은 $x$를 받습니다. 코드 실행을 통해 $J(.)$와 변화량(미분값) $\frac{\partial J}{\partial \theta}$을 계산합니다. 이제 gradient checking으로 $J$ 값에 대한 미분값들이 올바르게 계산됐는지 확인해봅시다.

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center> <u> **Figure 1** </u>: **1D linear model**<br> </center></caption>

위 그림은 계산의 핵심 단계를 보여줍니다:  
첫번째로 $x$를 받고 시작해 $J(x)$를 계산하는 순전파 과정을 거칩니다.  
두번째로 역전파 과정에 해당하는 $\frac{\partial J}{\partial \theta}$ 를 계산합니다.  

**Exercise**:   
위에서 설명한 간단한 방식의 순전파와 역전파를 구현해보세요  
좀 더 설명하자면 따로 따로 순전파 기능으로 $J(.)$를 계산하는 함수와 역전파 기능으로 $\theta$에 변화량을 구하는 함수를 만들어보세요


In [4]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(x, theta):
    """
    Implement the linear forward propagation (compute J) presented in Figure 1 (J(theta) = theta * x)
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    
    Returns:
    J -- the value of function J, computed using the formula J(theta) = theta * x
    """
    
    J = np.dot(theta, x)
    
    return J

In [5]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


**Expected Output**:

<table style=>
    <tr>
        <td>  ** J **  </td>
        <td> 8</td>
    </tr>
</table>

**Exercise**: 
이제 위 사진속 에서 역전파 부분을 구현할 시간입니다.

역전파를 구현하기 위해 매개변수 $\theta$에 대한 미분값을 구해야합니다.
구해진 값은 다음과 같이 저장합니다 $dtheta = \frac { \partial J }{ \partial \theta} = x$.


In [6]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(x, theta):
    """
    Computes the derivative of J with respect to theta (see Figure 1).
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    
    Returns:
    dtheta -- the gradient of the cost with respect to theta
    """
    
    dtheta = x
    
    return dtheta

In [7]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


**Expected Output**:

<table>
    <tr>
        <td>  ** dtheta **  </td>
        <td> 2 </td>
    </tr>
</table>

**Exercise**: 
`backward_propagation()` 기능으로 $\frac{\partial J}{\partial \theta}$를 올바르게 계산했는지 보여주기 위해 gradient checking을 수행합니다.


**Instructions**:
- 첫번째로 작은 $\varepsilon$를 사용해 "gradapprox"를 구합니다. 과정은 다음과 같습니다.:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
    
- 그 다음 역전파를 사용해 각 미분값들을 구하고 구해진 결과들을 "grad"에 저장합니다.

- 마지막으로 설명하는 수식을 통해 "gradapprox" 와 "grad" 의 차이를 확인해봅니다.:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$

위 식을 구하기 위해서는 다음 단계를 따라야합니다.:
   - 1'. 분자 부분을 계산하기 위해 np.linalg.norm(...)를 사용합니다.
   - 2'. 분모 부분을 계산하기 위해 np.linalg.norm(...)를 두번 사용해야합니다.
   - 3'. 이제 구한 값들을 각 위치에 자리하도록 나눠줍니다.

- 만약 차이가 작다면(작은 기준은  $10^{-7}$으로 삼습니다.) 계산된 미분값이 올바르다고 할 수 있습니다. 만약 좀 큰 값이라면 잘못된 계산이라고 할 수 있습니다.

In [8]:
# GRADED FUNCTION: gradient_check

def gradient_check(x, theta, epsilon=1e-7):
    """
    Implement the backward propagation presented in Figure 1.
    
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Compute gradapprox using left side of formula (1). epsilon is small enough, you don't need to worry about the limit.
    thetaplus = theta + epsilon                               # Step 1
    thetaminus = theta - epsilon                              # Step 2
    J_plus = forward_propagation(x, thetaplus)                # Step 3
    J_minus = forward_propagation(x, thetaminus)              # Step 4
    gradapprox = (J_plus - J_minus) / (2 * epsilon)           # Step 5
    
    # Check if gradapprox is close enough to the output of backward_propagation()
    grad = backward_propagation(x, theta)

    
    numerator = np.linalg.norm(grad - gradapprox)                      # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)    # Step 2'
    difference = numerator / denominator                               # Step 3'
    
    if difference < 1e-7:
        print("The gradient is correct!")
    else:
        print("The gradient is wrong!")
    
    return difference

In [9]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


**Expected Output**:
The gradient is correct!
<table>
    <tr>
        <td>  ** difference **  </td>
        <td> 2.9193358103083e-10 </td>
    </tr>
</table>

임계값 $10^{-7}$ 보다 작게 나오네요!! 따라서 올바르게  `backward_propagation()`를 구현했다고 할 수 있습니다.


이제 gradient checking이 어떤식으로 작동하는지 알게됐습니다. 이지 좀 더 일반적인 케이스에 적용해봅시다.

딥러닝에서 비용함수 $J$를 구하는데 여러 매개변수를 사용합니다.
일반적으로 신경망을 학습시킬때는 $\theta$를 다차원인 $W^{[l]}$와  $b^{[l]}$를 사용합니다

입력값이 다차원일때 gradinet checking을 구현하는게 더 중요합니다. 한번 해보죠

## 3) N-차원 gradient checking

아래 사진은 사기 거래를 감지하는 모델의 순전파와 역전파를 표현했습니다.

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: **deep neural network**<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

In [10]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation (and computes the cost) presented in Figure 3.
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (5, 4)
                    b1 -- bias vector of shape (5, 1)
                    W2 -- weight matrix of shape (3, 5)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    
    Returns:
    cost -- the cost function (logistic cost for one example)
    """
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

역전파를 구현단계입니다.

In [11]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

어느정도 검정데이터에 대한 결과를 얻었습니다. 하지만 모델의 성능에 대해 100% 확신을 못하죠.

완벽한건 없습니다. gradient checking을 통해 올바른지 확인해봅시다.

**gradient checking 이 어떻게 작동하나요?**.

앞에서 했던 방법과 같이 "gradapprox" 와 변화량 값(gradient)를 비교해야합니다. 
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

하지만 더이상 $\theta$가 단순히 한개의 실수값이 아닙니다.
여러개의 매개변수가 있고 각 매개변수는 사이즈가 제각각입니다. 간단하게 하기 위해 매개변수들을 한 변수에 저장시키는 방법을 사용할겁니다.

편의를 위해 매개변수저장은 모듈로 `dictionary_to_vector()` 미리 만들어놨습니다.

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: **dictionary_to_vector() and vector_to_dictionary()**<br> You will need these functions in gradient_check_n()</center></caption>

그리고 계산된 "gradients" 또한 dictionary에 저장시키기 위해 `gradients_to_vector()`를 사용할거니 걱정할 필요없습니다. 

**Exercise**: gradient_check_n() 실행.

**Instructions**: gradient check를 구현하기 위한 예시용 코드입니다.

각 매개변수에 대해 계산하는 방법:
- `J_plus[i]`계산 하는 방법:
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- To compute `J_minus[i]`: do the same thing with $\theta^{-}$
- Compute $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

Thus, you get a vector gradapprox, where gradapprox[i] is an approximation of the gradient with respect to `parameter_values[i]`. You can now compare this gradapprox vector to the gradients vector from backpropagation. Just like for the 1D case (Steps 1', 2', 3'), compute: 
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [14]:
# GRADED FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output by forward_propagation_n
    
    Arguments:
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
    grad -- output of backward_propagation_n, contains gradients of the cost with respect to the parameters. 
    x -- input datapoint, of shape (input size, 1)
    y -- true "label"
    epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
    
    Returns:
    difference -- difference (2) between the approximated gradient and the backward propagation gradient
    """
    
    # Set-up variables
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # Compute gradapprox
    for i in range(num_parameters):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        # "_" is used because the function you have to outputs two parameters but we only care about the first one
        ### START CODE HERE ### (approx. 3 lines)
        thetaplus =  np.copy(parameters_values)                                       # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                   # Step 2
        J_plus[i], _ =  forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))  # Step 3
        ### END CODE HERE ###
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        ### START CODE HERE ### (approx. 3 lines)
        thetaminus = np.copy(parameters_values)                                       # Step 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                 # Step 2        
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus)) # Step 3
        ### END CODE HERE ###
        
        # Compute gradapprox[i]
        ### START CODE HERE ### (approx. 1 line)
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
        ### END CODE HERE ###
    
    # Compare gradapprox to backward propagation gradients by computing difference.
    ### START CODE HERE ### (approx. 1 line)
    numerator = np.linalg.norm(grad - gradapprox)                                     # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                   # Step 2'
    difference = numerator / denominator                                              # Step 3'
    ### END CODE HERE ###

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [15]:
X, Y, parameters = gradient_check_n_test_case()
#print(parameters)

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

{'W1': array([[-0.3224172 , -0.38405435,  1.13376944, -1.09989127],
       [-0.17242821, -0.87785842,  0.04221375,  0.58281521],
       [-1.10061918,  1.14472371,  0.90159072,  0.50249434],
       [ 0.90085595, -0.68372786, -0.12289023, -0.93576943],
       [-0.26788808,  0.53035547, -0.69166075, -0.39675353]]), 'b1': array([[-0.6871727 ],
       [-0.84520564],
       [-0.67124613],
       [-0.0126646 ],
       [-1.11731035]]), 'W2': array([[ 0.2344157 ,  1.65980218,  0.74204416, -0.19183555, -0.88762896],
       [-0.74715829,  1.6924546 ,  0.05080775, -0.63699565,  0.19091548],
       [ 2.10025514,  0.12015895,  0.61720311,  0.30017032, -0.35224985]]), 'b2': array([[-1.1425182 ],
       [-0.34934272],
       [-0.20889423]]), 'W3': array([[0.58662319, 0.83898341, 0.93110208]]), 'b3': array([[0.28558733]])}
There is a mistake in the backward propagation! difference = 0.2850931566540251


**Expected output**:

<table>
    <tr>
        <td>  ** There is a mistake in the backward propagation!**  </td>
        <td> difference = 0.285093156781 </td>
    </tr>
</table>

It seems that there were errors in the `backward_propagation_n` code we gave you! Good that you've implemented the gradient check. Go back to `backward_propagation` and try to find/correct the errors *(Hint: check dW2 and db1)*. Rerun the gradient check when you think you've fixed it. Remember you'll need to re-execute the cell defining `backward_propagation_n()` if you modify the code. 

Can you get gradient check to declare your derivative computation correct? Even though this part of the assignment isn't graded, we strongly urge you to try to find the bug and re-run gradient check until you're convinced backprop is now correctly implemented. 

**Note** 
- Gradient Checking is slow! Approximating the gradient with $\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ is computationally costly. For this reason, we don't run gradient checking at every iteration during training. Just a few times to check if the gradient is correct. 
- Gradient Checking, at least as we've presented it, doesn't work with dropout. You would usually run the gradient check algorithm without dropout to make sure your backprop is correct, then add dropout. 

Congrats, you can be confident that your deep learning model for fraud detection is working correctly! You can even use this to convince your CEO. :) 

<font color='blue'>
**What you should remember from this notebook**:
- Gradient checking verifies closeness between the gradients from backpropagation and the numerical approximation of the gradient (computed using forward propagation).
- Gradient checking is slow, so we don't run it in every iteration of training. You would usually run it only to make sure your code is correct, then turn it off and use backprop for the actual learning process. 